# Main programme avec interaction avec l'utilisateur



L'idée de ce dossier est de modéliser un CP avec interaction d'un décision-maker.

In [70]:
#############################
### Import libraries ###
#############################

from docplex.cp.model import *
from docplex.cp.config import get_default
import numpy as np
from Solver import *
from User import *
import time

In [71]:
#############################
### Main program ###
#############################

print("\n--------Main program is loading...---------")

# Parameters

k = 20

# Interaction with the solver

data = []
file = './file_with_optimal_val/la03.txt'
optimalval = -1

with open(file, 'r') as f:    
    for line in f:
        if line.startswith('optimalval ='):
            line_splitted = line.strip().split('=')
            optimalval = int(line_splitted[1])
        else:
            data.append(line.strip().split())

n = int(data[0][0])
m = int(data[0][1])

T_machine = []
T_duration = []

for i in range(1,n+1):
    for j in range(0, 2*m, 2):
        T_machine.append(int(data[i][j]))
        T_duration.append(int(data[i][j+1]))

duration = np.zeros((n, m))

for i in range(n):
    for j in range(m):
        ind_machine = int(T_machine[i*m + j])
        duration[i][ind_machine] = T_duration[i*m + j]


# --------- Call Solver constructor in Solver.py

solver = Solver(data)

model = CpoModel() 

# --------- Create the model variables 

solver.create_variables(model, n, m, duration)

# ------------ Solve the model

print("\nSolving the model...")
msol, nb_solution, runtime = solver.solve(model, k, n, m, T_machine, optimalval)

# ------------ Display the result

# for sol in msol:
#     sol.write()
print("Model solved !")


# ---------------- Interaction with the user

print("\n--------Interaction with the user...---------")

print("\nCreating the user...")
user = User()
print("User created !")

print("\nClassing solutions...")	
list_indice, list_equal, list_obj = user.classerSolutions(msol)
# print(list_obj)
print("Solutions classed !")

print("\nCreating preferences...")
pref = user.getPreferences()
print("Preferences created !")

print("\nDisplaying preferences...")
# for sol in pref:
#     sol.write()

# print("\nPreferences : ", pref)
# print(list_indice)
# print(list_equal)

variables = solver.get_variables()
matrix = user.matrix_pref(n, m, False)

pref = user.getPreferences()

print("\nTesting preferences...")

if user.test_preferences(pref):
    print("\tL'ordre des préférences est cohérente")
else:
    print("\tL'ordre des préférences n'est pas cohérente")
    
if user.test_differences_sol(matrix):
    print("\tToutes les solutions sont différentes")
else:
    print("\tLes solutions ne sont pas toutes différentes")



--------Main program is loading...---------

Solver initialized !

Creating the model variables...
Model variables created !

Solving the model...

Adding precedence constraints to the solver...
Precedence constraints added !

Adding disjunctive constraints to the solver...
Disjunctive constraints added !
Model solved !

--------Interaction with the user...---------

Creating the user...
User created !

Classing solutions...
Solutions classed !

Creating preferences...
Preferences created !

Displaying preferences...

Testing preferences...
	L'ordre des préférences est cohérente
	Toutes les solutions sont différentes


In [72]:
def abs_int(x,y):
    return max(x-y,y-x)

def abs_(L1,L2):
    L=[]
    for i in range(len(L1)):
        L.append(abs_int(L1[i],L2[i]))
    return L

def sum_(L):
    s=0
    for i in range(len(L)):
        s+=L[i]
    return s

def manhattan_distance(sol1, sol2):
    sol1np=np.array(sol1)
    sol2np=np.array(sol2)
    return sum_(abs_(sol1np,sol2np))
# def manhattan_distance_siala(self,sol1,sol2):
#     return np.sum(np.abs(sol1-sol2))

def rayon_cluster(avg, list_sol):
    rayon = 0
    for sol in list_sol:
        rayon = max(rayon, manhattan_distance(avg, sol))
    return rayon


In [73]:
############################################################
#### AJOUT DES CLUSTER A CHAQUE ITERATION
############################################################

import clustering as cl

###  -------------- Iteration of the solver with the preferences

# ----------------- Parameters

tps_max = 100
tps = runtime
it_max = 5
it = 1
k_k = 10

criterion = (tps < tps_max) and (it < it_max) 

# ----------------- Add the preferences to the model

while criterion :

    print("\n--------Iteration {}---------".format(it))

    it += 1
    
    solver = Solver(data)
    model = CpoModel() 

    # --------- Create the model variables 

    solver.create_variables(model, n, m, duration) 

    # stop = int(input("Do you want to continue ? True(1) or False(0)"))

    bb = integer_var(0,1)
    solver.add_variable(bb)
    bb=1

    list_variables = solver.get_variables()
    variables = list_variables[0]

    for sol in pref:
        b = integer_var(0,1)
        solver.add_variable(b)
        b=0

        for i in range(n):
            for j in range(m):

                var_sol = sol.get_value("T{}-{}".format(i,j))
                #a = model.interval_var(start = sol[variables[i][j]].start, end= sol[variables[i][j]].end, size=int(duration[i][j]), name="a{}{}".format(i,j))
                #b =max(b,logical_or((model.start_of(a) != model.start_of(variables[i][j])), (model.end_of(a) != model.end_of(variables[i][j]))))
                
                b =max(b,logical_or(var_sol.start != model.start_of(variables[i][j]), var_sol.end != model.end_of(variables[i][j])))
        b = (b!=0)
        bb = bb * b
    solver.add_constraint(bb==1)
    

# ------------ Solve the model

    print("\nSolving the model...")

    msol, nb_solution, runtime = solver.solve(model, k_k, n, m, T_machine, optimalval)

# ------------ Display the result

    # for sol in msol:
    #     sol.write()
    print("Model solved !")


# ---------------- Interaction with the user

    print("\n--------Interaction with the user...---------")

    # print("\nClassing solutions...")	
    list_indice, list_equal, list_obj = user.classerSolutions(msol)
    print(list_obj)
    print(list_obj[0])
    # print(list_indice)
    # print(list_equal)
    # print(list_indice)
    # print("Solutions classed !")

    # print("\nCreating preferences...")
    pref = user.getPreferences()
    # print("Preferences created !")

    # print("\nDisplaying preferences...")
    # for sol in pref:
    #     sol.write()

    # print("\nPreferences : ", pref)
    # print(list_indice)
    # print(list_equal)
    # print(len(pref))

    starts = user.start_pref(n, m, variables, False)

    matrix = user.matrix_pref(n, m, False)
    # print(len(matrix))
    # print("\nTesting preferences...")

    pref = user.getPreferences()
    
    if user.test_preferences(pref):
        print("\tL'ordre des préférences est cohérente")
    else:
        print("\tL'ordre des préférences n'est pas cohérente")
        
    if user.test_differences_sol(matrix):
        print("\tToutes les solutions sont différentes")
    else:
        print("\tLes solutions ne sont pas toutes différentes")


        
    layers = cl.create_layers(list_equal, matrix)
    # print(layers)
    print(len(layers))

    nb_clusters = []

    for i in range(len(layers)):
        data = layers[i]
        # print("-------------- Layer", i, "-----------------")
        # print(len(data))
        
        if len(data) > 2:
            list, k_max, leaves_max, runtime = cl.silhouette(data)
            nb_clusters.append(k_max)

            # print(len(data))
            # print("D'après l'indice de silhouette, nb clusters =", k_max ,", nb feuilles = ", leaves_max , 
            #     " runtime = ", runtime ,"s \n")
            
        else:
        #     print("Pas assez de données pour faire un clustering : On a juste ", len(data), " données")
        #     print("numbre de clusters = ", len(data),"\n")
            nb_clusters.append(len(data))

        


    default_value_rayon = 3     # should be a parameter
    
    for l in range(len(layers)):
        data = layers[l]
        # print("-------------- Layer", l, "-----------------")
        if(len(data)==1):
            solver.add_constraint(manhattan_distance(data[0], [ model.start_of(variables[i//m][i%m]) for i in range(n*m)]) >= default_value_rayon + l)
            continue
        
        k = nb_clusters[l]

        k , leaves , labels, runtime = cl.my_agglo_k(data, k, 'single')
        print(k,labels)

        avg=[[0 for i in range(len(data[0]))] for i in range(k)]
        card=[0 for i in range(k) ]

        for i in range(len(data)):
            card[labels[i]]+=1
            avg[labels[i]]=[avg[labels[i]][k]+data[i][k] for k in range(len(data[i]))]
        
        for i in range(k):
            avg[i]=[avg[i][k]/card[i] for k in range(len(data[0]))]


        
        the_ones=[None for i in range(k)]
        for i in range(len(data)):
            if(the_ones[labels[i]]==None or manhattan_distance(data[i],avg[labels[i]])<manhattan_distance(the_ones[labels[i]],avg[labels[i]])):
                    the_ones[labels[i]]=data[i]

        #ajout de contraintes, les solutions doivent etre distantes des centres de clusters, en chaque layer , d'une distance supérieure à 3
        for i in range(k):
            sol_cluster=[]
            for j in range(len(data)):
                if labels[j]==i:
                    sol_cluster.append(data[j])
            solver.add_constraint(manhattan_distance(the_ones[i],[ model.start_of(variables[i//m][i%m]) for i in range(n*m)]) >= rayon_cluster(the_ones[i], sol_cluster) + l)
    

    tps += runtime

    criterion = (tps < tps_max) and (it < it_max)

    if (it == it_max):
        print("The user has reached the maximum number of iterations !")
    # elif not stop:
    #     print("The user has chosen his/her solution !")
    elif (tps >= tps_max):
        print("The user has reached the maximum time !")



--------Iteration 1---------

Solver initialized !

Creating the model variables...
Model variables created !

Solving the model...

Adding precedence constraints to the solver...
Precedence constraints added !

Adding disjunctive constraints to the solver...
Disjunctive constraints added !
Model solved !

--------Interaction with the user...---------
[735, 734, 734, 728, 696, 699, 726, 724, 728, 735, 696, 696, 696, 696, 696, 696, 696, 702, 702, 702, 707, 710, 712, 715, 726, 728, 729, 735, 773, 778]
735
	L'ordre des préférences est cohérente
	Toutes les solutions sont différentes
Clustering ...
15
2 [0 1 1 0 1 1 1 0]
2 [1 0 0]
2 [1 0]
2 [1 0 0]
2 [1 0]
2 [0 1 0]

--------Iteration 2---------

Solver initialized !

Creating the model variables...
Model variables created !

Solving the model...

Adding precedence constraints to the solver...
Precedence constraints added !

Adding disjunctive constraints to the solver...
Disjunctive constraints added !
Model solved !

--------Interaction